In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns

import processing

## Lab participants

In [2]:
%%time
lab_log = processing.get_lab_results()

/Users/itto/miniconda3/lib/python3.6/site-packages/IPython/core/magics/execution.py:1238: DtypeWarning: Columns (20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)
/Users/itto/miniconda3/lib/python3.6/site-packages/IPython/core/magics/execution.py:1238: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


CPU times: user 42.7 s, sys: 6.06 s, total: 48.7 s
Wall time: 49 s


## Aggregate words

In [60]:
ts = processing.get_test_sections()

/Users/itto/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [904]:
a = lab_log.copy()

### 0. Preprocess

In [1148]:
a.loc[a.key.isna(),'key'] = '_'

# Replace empty text fields
a.loc[a.text_field.isna(),'text_field'] = ''

In [1149]:
a['text_field_prev'] = a.text_field.shift(1)

In [1150]:
a.lev_dist = a.lev_dist.astype('int')

### 1. Designate forward entries

In [1151]:
# Default
a['is_forward'] = False

#### 1a.  defined keys

In [1152]:
%%time
mask = a.key != 'undefined'
a.loc[mask,'is_forward'] = a.loc[mask].apply(lambda x: x.text_field[-len(x.key):] == x.key, axis=1)

CPU times: user 224 ms, sys: 3.75 ms, total: 228 ms
Wall time: 229 ms


#### 1b. undefined keys

In [1153]:
%%time
# Case 1: zero LD. We assume this is always forward.
mask = (a.key == 'undefined') & (a.lev_dist == 0)
a.loc[mask,'key'] = ''
a.loc[mask,'is_forward'] = True

# Case 2: Beginning of a sentence
first_key = a.groupby('ts_id').head(1).text_field
a.loc[first_key.index,'key'] = first_key
a.loc[first_key.index,'is_forward'] = True

# Case 3: Fulfills preliminary forward conditions
def find_key(x):
    # Check that all of LD is due to additions at the end of the text field
    if x.text_field[:-x.lev_dist] == x.text_field_prev:
        return x.text_field[-x.lev_dist:],True
    return 'undefined',False
# Condition: Positive LD, and LD is accounted for by a difference in length
mask = (a.key == 'undefined') & (a.lev_dist > 0) & (a.len_diff == a.lev_dist)
key,is_forward = list(zip(*a.loc[mask].apply(find_key,axis=1)))
a.loc[mask,'key'] = key
a.loc[mask, 'is_forward'] = is_forward

CPU times: user 19.9 ms, sys: 2.14 ms, total: 22 ms
Wall time: 19.9 ms


### 2. Process forward entries

In [1224]:
b = a[['ts_id','key','text_field','is_forward','lev_dist','len_diff','iki','ite']].copy()
b.reset_index(drop=True,inplace= True)

In [1225]:
b.head(10)

,ts_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite
0,1626925.0,A,A,True,49,NaN,NaN,none
1,1626925.0,Ar,Ar,True,1,1.0,684.0,none
2,1626925.0,Ark,Ark,True,1,1.0,164.0,none
3,1626925.0,Arka,Arka,True,1,1.0,797.0,none
4,1626925.0,Arkan,Arkan,True,1,1.0,147.0,none
5,1626925.0,Arkans,Arkans,True,1,1.0,269.0,none
6,1626925.0,Arkansa,Arkansa,True,1,1.0,247.0,none
7,1626925.0,Arkansas,Arkansas,True,1,1.0,293.0,none
8,1626925.0,Arkansas,Arkansas,True,0,0.0,171.0,none
9,1626925.0,,Arkansas,True,1,1.0,6.0,none


In [1226]:
b['is_rep'] = False

# Case 1: A multi-key entry is repeated quickly
mask = (b.key.shift(-1) == b.key) & (b.key.shift(-1).str.len() > 1) & (b.lev_dist.shift(-1) == 0) & (b.iki.shift(-1) < 30)
b.loc[mask,'iki'] += b.shift(-1).loc[mask,'iki']
b.loc[mask,'is_rep'] = True
b.drop(b.loc[mask].index + 1, inplace=True)
b.reset_index(drop=True,inplace=True)

# Case 2: Empty key
mask = (b.key.shift(-1) == '')
b.loc[mask,'iki'] += b.shift(-1).loc[mask,'iki']
b.loc[mask,'is_rep'] = True
b.drop(b.loc[mask].index + 1, inplace=True)
b.reset_index(drop=True,inplace=True)

In [1227]:
b.head(10)

,ts_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep
0,1626925.0,A,A,True,49,NaN,NaN,none,False
1,1626925.0,Ar,Ar,True,1,1.0,684.0,none,False
2,1626925.0,Ark,Ark,True,1,1.0,164.0,none,False
3,1626925.0,Arka,Arka,True,1,1.0,797.0,none,False
4,1626925.0,Arkan,Arkan,True,1,1.0,147.0,none,False
5,1626925.0,Arkans,Arkans,True,1,1.0,269.0,none,False
6,1626925.0,Arkansa,Arkansa,True,1,1.0,247.0,none,False
7,1626925.0,Arkansas,Arkansas,True,1,1.0,293.0,none,False
8,1626925.0,Arkansas,Arkansas,True,0,0.0,171.0,none,False
9,1626925.0,,Arkansas,True,1,1.0,6.0,none,False


In [1228]:
b['space_added'] = 'none'

# Case 1: merge quick spaces with the previous entry
mask = (b.key.shift(-1) == ' ') & (b.iki.shift(-1) < 30)
b.loc[mask,['key','lev_dist','len_diff','iki']] += b.shift(-1).loc[mask,['key','lev_dist','len_diff','iki']]
b.loc[mask,'space_added'] = 'trailing'
b.drop(b.loc[mask].index + 1, inplace=True)
b.reset_index(drop=True,inplace=True)

# Case 2: merge slow spaces with the following entry (if the following is more than one char, which suggests swype)
mask = (b.key.shift(1) == ' ') & (b.iki.shift(1) > 30) & (b.key.str.len() > 1)
b.loc[mask,['key','lev_dist','len_diff','iki']] += b.shift(1).loc[mask,['key','lev_dist','len_diff','iki']]
b.loc[mask,'space_added'] = 'leading'
b.drop(b.loc[mask].index - 1, inplace=True)
b.reset_index(drop=True,inplace=True)

# Case 3: already leading space with multiple characters following the space
mask = (b.key.str[0] == ' ') & (b.space_added == 'none') & (b.key.str.len() > 2)
b.loc[mask,'space_added'] = 'leading'

# Case 4: already leading space but only one character following the space
# Do nothing. This is not actually a leading space
pass

In [1229]:
b.head(10)

,ts_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep,space_added
0,1626925.0,A,A,True,49.0,NaN,NaN,none,False,none
1,1626925.0,Ar,Ar,True,1.0,1.0,684.0,none,False,none
2,1626925.0,Ark,Ark,True,1.0,1.0,164.0,none,False,none
3,1626925.0,Arka,Arka,True,1.0,1.0,797.0,none,False,none
4,1626925.0,Arkan,Arkan,True,1.0,1.0,147.0,none,False,none
5,1626925.0,Arkans,Arkans,True,1.0,1.0,269.0,none,False,none
6,1626925.0,Arkansa,Arkansa,True,1.0,1.0,247.0,none,False,none
7,1626925.0,Arkansas,Arkansas,True,1.0,1.0,293.0,none,False,none
8,1626925.0,Arkansas,Arkansas,True,1.0,1.0,177.0,none,False,trailing
9,1626925.0,c,Arkansas c,True,1.0,1.0,142.0,none,False,none


In [1230]:
# Entry id based on the number of separators
b['contains_sep'] = b.key.str.contains(' |_') # Dummy
b['entry_id'] = b.groupby('ts_id').contains_sep.cumsum()
b.drop('contains_sep',axis=1,inplace=True) # Remove dummy

In [1231]:
b.head(15)

,ts_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep,space_added,entry_id
0,1626925.0,A,A,True,49.0,NaN,NaN,none,False,none,0.0
1,1626925.0,Ar,Ar,True,1.0,1.0,684.0,none,False,none,0.0
2,1626925.0,Ark,Ark,True,1.0,1.0,164.0,none,False,none,0.0
3,1626925.0,Arka,Arka,True,1.0,1.0,797.0,none,False,none,0.0
4,1626925.0,Arkan,Arkan,True,1.0,1.0,147.0,none,False,none,0.0
5,1626925.0,Arkans,Arkans,True,1.0,1.0,269.0,none,False,none,0.0
6,1626925.0,Arkansa,Arkansa,True,1.0,1.0,247.0,none,False,none,0.0
7,1626925.0,Arkansas,Arkansas,True,1.0,1.0,293.0,none,False,none,0.0
8,1626925.0,Arkansas,Arkansas,True,1.0,1.0,177.0,none,False,trailing,1.0
9,1626925.0,c,Arkansas c,True,1.0,1.0,142.0,none,False,none,1.0


In [1232]:
first_instance = b.loc[~b.is_forward].groupby(['ts_id','entry_id']).head(1)
total_time = b.loc[~b.is_forward].groupby(['ts_id','entry_id']).iki.sum()

b.loc[first_instance.index,'key'] = '_'
b.loc[first_instance.index,'iki'] = total_time.values

# Remove the rest
mask = (~b.index.isin(first_instance.index)) & (~b.is_forward)
b.drop(b.loc[mask].index,inplace=True)

In [1233]:
# Now that we condensed the non-forward entries, reassign the entry ids
b['contains_sep'] = b.key.str.contains(' |_') # Dummy
b['entry_id'] = b.groupby('ts_id').contains_sep.cumsum()
b.drop('contains_sep',axis=1,inplace=True) # Remove dummy

In [1234]:
b.loc[50:60]

,ts_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep,space_added,entry_id
50,1626925.0,pris,Arkansas Caledonian in 32 extra state police t...,True,1.0,1.0,183.0,none,False,none,9.0
51,1626925.0,priso,Arkansas Caledonian in 32 extra state police t...,True,1.0,1.0,87.0,none,False,none,9.0
52,1626925.0,prison,Arkansas Caledonian in 32 extra state police t...,True,1.0,1.0,238.0,none,False,none,9.0
53,1626925.0,prison,Arkansas Caledonian in 32 extra state police t...,True,0.0,0.0,352.0,none,False,none,9.0
54,1626925.0,.,Arkansas Caledonian in 32 extra state police t...,True,1.0,1.0,5.0,none,False,none,9.0
55,1626925.0,_,Arkansas Caledonia in 32 extra state police to...,False,1.0,-1.0,2875.0,none,False,none,10.0
56,1626925.0,_,Arkansas Caledoni in 32 extra state police to ...,False,1.0,-1.0,416.0,none,False,none,11.0
57,1626925.0,_,Arkansas Caledon in 32 extra state police to t...,False,1.0,-1.0,63.0,none,False,none,12.0
58,1626925.0,_,Arkansas Caledo in 32 extra state police to th...,False,1.0,-1.0,58.0,none,False,none,13.0
59,1626925.0,_,Arkansas Caled in 32 extra state police to the...,False,1.0,-1.0,59.0,none,False,none,14.0


In [1235]:
# Case 1: Keys with trailing spaces are actually part of the new entry id
b.loc[b.space_added == 'trailing','entry_id'] -= 1

# Case 2: Keys that are not trailing/leading have undefined entry
b.loc[(b.space_added == 'none') & (b.key == ' '),'entry_id'] = -1
b.loc[(b.space_added == 'none') & (b.key == '_'),'entry_id'] = -1

# Case 3: Keys with leading spaces are unchanged
pass

In [1236]:
b.head(10)

,ts_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep,space_added,entry_id
0,1626925.0,A,A,True,49.0,NaN,NaN,none,False,none,0.0
1,1626925.0,Ar,Ar,True,1.0,1.0,684.0,none,False,none,0.0
2,1626925.0,Ark,Ark,True,1.0,1.0,164.0,none,False,none,0.0
3,1626925.0,Arka,Arka,True,1.0,1.0,797.0,none,False,none,0.0
4,1626925.0,Arkan,Arkan,True,1.0,1.0,147.0,none,False,none,0.0
5,1626925.0,Arkans,Arkans,True,1.0,1.0,269.0,none,False,none,0.0
6,1626925.0,Arkansa,Arkansa,True,1.0,1.0,247.0,none,False,none,0.0
7,1626925.0,Arkansas,Arkansas,True,1.0,1.0,293.0,none,False,none,0.0
8,1626925.0,Arkansas,Arkansas,True,1.0,1.0,177.0,none,False,trailing,0.0
9,1626925.0,c,Arkansas c,True,1.0,1.0,142.0,none,False,none,1.0


In [1237]:
b = b.loc[~b.key.str.contains('.',regex=False)].copy()
b = b.loc[~b.key.str.contains(',',regex=False)].copy()
b.reset_index(drop = True,inplace= True)

In [1248]:
b['iki_norm'] = b.iki / b.key.str.len()

In [1249]:
b.groupby('ite').iki.describe()

,count,mean,std,min,25%,50%,75%,max
ite,,,,,,,,
autocorr,43.0,351.534884,396.042647,30.0,83.50,198.0,438.0,1400.0
none,4310.0,360.960325,435.423014,3.0,156.00,223.0,403.0,6385.0
predict,66.0,748.909091,442.630241,17.0,559.25,680.0,931.0,2181.0
swype,407.0,943.771499,732.107353,4.0,386.00,763.0,1184.5,4333.0


In [1250]:
b.groupby('ite').iki_norm.describe()

,count,mean,std,min,25%,50%,75%,max
ite,,,,,,,,
autocorr,43.0,82.326024,112.966900,5.900000,19.619048,37.125000,111.166667,609.0
none,4310.0,326.456410,419.682931,0.800000,115.500000,201.000000,384.750000,6385.0
predict,66.0,122.842407,114.615026,5.666667,77.129464,102.883333,132.025000,621.0
swype,407.0,205.072202,208.123983,0.571429,85.333333,176.250000,265.125000,3110.0


### Assign ITE

In [1260]:
c = b.copy()

In [1261]:
c['ite'] = 'none'

#### 1. Swype

In [1262]:
# Case 1: Has leading spaces
c.loc[c.space_added == 'leading','ite'] = 'swype'

# Case 2: The first action of the first entry has multiple characters (not including spaces)
index_first = c.loc[c.entry_id == 0].groupby(['ts_id']).head(1).index
mask = (c.index.isin(index_first)) & (c.key.str.strip(' ').str.len() > 1)
c.loc[mask,'ite'] = 'swype'

# Case 2: The first action of an entry has multiple characters (not including spaces)
index_first = c.groupby(['ts_id','entry_id']).head(1).index
mask = (c.index.isin(index_first)) & (c.key.str.strip(' ').str.len() > 1) & (c.iki_norm > 130)
c.loc[mask,'ite'] = 'swype'

# Case 3: Fill in the same entry as a swype
c.set_index(['ts_id','entry_id'],inplace=True)
c.loc[c.loc[c.ite == 'swype'].index,'ite'] = 'swype'
c.reset_index(inplace=True)

# TODO Case 2 could also mean a prediction
# TODO what about swype followed by a prediction correction?
# TODO what about backspace followed by a prediction?



In [1263]:
c.loc[(c.ite == 'swype') & (b.ite != 'swype')].shape

(131, 12)

In [1264]:
c.loc[(c.ite != 'swype') & (b.ite == 'swype')]

,ts_id,entry_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep,space_added,iki_norm
3019,1735919.0,2.0,soffit,They are soffit,True,6.0,6.0,9.0,none,False,none,1.500000
3027,1735919.0,8.0,more,They are specifically concerned about the inte...,True,4.0,4.0,5.0,none,False,none,1.250000
3032,1735925.0,2.0,know,Let me know,True,5.0,5.0,512.0,none,True,trailing,102.400000
3036,1735925.0,6.0,anything,Let me know if you need anything,True,9.0,9.0,826.0,none,True,trailing,91.777778
3043,1735929.0,5.0,going,The evacuees included backs and going,True,5.0,5.0,6.0,none,False,none,1.200000
3044,1735929.0,-1.0,_,The evacuees included backs and young children,False,11.0,9.0,3110.0,none,True,none,3110.000000
3047,1735933.0,2.0,should,What number should,True,7.0,7.0,410.0,none,True,trailing,58.571429
3049,1735933.0,4.0,call,What number should be call,True,5.0,5.0,640.0,none,True,trailing,128.000000
3055,1735937.0,1.0,didn't,It didn't,True,6.0,6.0,7.0,none,False,none,1.166667
3060,1735937.0,4.0,spreadsheet,It doesn't match your spreadsheet,True,11.0,11.0,1283.0,none,True,none,116.636364


#### 2. Predict

In [1266]:
# Case 1: The last action of an entry has multiple characters, LD > 1
index_last = c.groupby(['ts_id','entry_id']).tail(1).index
mask = (c.index.isin(index_last)) & (c.key.str.strip(' ').str.len() > 1)
mask &= (c.lev_dist > 1) & (c.ite != 'swype') & (c.iki > 400)
c.loc[mask,'ite'] = 'predict'

# Case 2: The last action of an entry has multiple characters, LD == 1
index_last = c.groupby(['ts_id','entry_id']).tail(1).index
mask = (c.index.isin(index_last)) & (c.key.str.strip(' ').str.len() > 1)
mask &= (c.lev_dist == 1) & (c.space_added == 'none') & (c.iki > 400)
c.loc[mask,'ite'] = 'predict'

# # Case 3: Fill in the same entry as a predict
# c.set_index(['ts_id','entry_id'],inplace=True)
# c.loc[c.loc[c.ite == 'predict'].index,'ite'] = 'predict'
# c.reset_index(inplace=True)

In [1277]:
c.loc[(c.ite == 'predict') & (b.ite != 'predict')].shape

(43, 12)

In [1278]:
c.loc[(c.ite != 'predict') & (b.ite == 'predict')].shape

(17, 12)

In [1279]:
c.groupby('ite').iki.describe()

,count,mean,std,min,25%,50%,75%,max
ite,,,,,,,,
none,4232.0,353.430057,419.574135,3.0,155.00,221.0,398.00,6385.0
predict,92.0,795.282609,273.220811,410.0,606.25,698.5,941.75,1771.0
swype,502.0,867.561753,778.979538,5.0,333.25,679.5,1159.50,5865.0


In [1280]:
c.groupby('ite').iki_norm.describe()

,count,mean,std,min,25%,50%,75%,max
ite,,,,,,,,
none,4232.0,329.444996,423.933226,0.571429,117.000,202.00,388.2500,6385.00
predict,92.0,117.978172,49.044410,41.333333,83.625,111.90,131.2250,332.75
swype,502.0,193.374330,176.986208,0.833333,52.375,170.75,269.4375,1734.00


In [1290]:
c.loc[c.ts_id == c.loc[c.text_field.str.contains('s working',regex=False),'ts_id'].iloc[0]]

,ts_id,entry_id,key,text_field,is_forward,lev_dist,len_diff,iki,ite,is_rep,space_added,iki_norm
3092,1735951.0,0.0,I,I,True,40.0,NaN,NaN,none,True,trailing,NaN
3093,1735951.0,1.0,know,I know,True,5.0,5.0,988.0,swype,True,trailing,197.600000
3094,1735951.0,2.0,he's,I know he's,True,5.0,5.0,1974.0,swype,True,trailing,394.800000
3095,1735951.0,3.0,working,I know he's working,True,8.0,8.0,1037.0,predict,True,trailing,129.625000
3096,1735951.0,4.0,on,I know he's working on,True,3.0,3.0,562.0,swype,True,trailing,187.333333
3097,1735951.0,5.0,it,I know he's working on it,True,2.0,2.0,852.0,swype,True,none,426.000000


In [1285]:
lab_log.loc[lab_log.ts_id == 1735937.0]

,log_data_id,ts_id,type,key,text_field,timestamp,input_len,lev_dist,input_len_prev,lev_dist_prev,swype,predict,autocorr,iki,iki_norm,len_diff,ite,participant_id
3661,229145980,1735937.0,input,undefined,It,1.547554e+12,2.0,33.0,34.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,swype,265900
3662,229145981,1735937.0,input,undefined,It,1.547554e+12,2.0,0.0,2.0,33.0,0.0,0.0,0.0,1367.0,inf,0.0,none,265900
3663,229145983,1735937.0,input,undefined,It,1.547554e+12,3.0,1.0,2.0,0.0,0.0,0.0,0.0,5.0,5.000000,1.0,none,265900
3664,229145985,1735937.0,input,undefined,It didn't,1.547554e+12,9.0,6.0,3.0,1.0,1.0,0.0,0.0,7.0,1.166667,6.0,swype,265900
3665,229145986,1735937.0,input,undefined,It,1.547554e+12,3.0,6.0,9.0,6.0,0.0,0.0,0.0,1375.0,229.166667,-6.0,none,265900
3666,229145987,1735937.0,input,undefined,It doesn't,1.547554e+12,10.0,7.0,3.0,6.0,0.0,0.0,0.0,6.0,0.857143,7.0,none,265900
3667,229145989,1735937.0,input,undefined,It doesn't,1.547554e+12,11.0,1.0,10.0,7.0,0.0,0.0,0.0,1451.0,1451.000000,1.0,none,265900
3668,229145991,1735937.0,input,undefined,It doesn't match,1.547554e+12,16.0,5.0,11.0,1.0,1.0,0.0,0.0,5.0,1.000000,5.0,swype,265900
3669,229145992,1735937.0,input,undefined,It doesn't match,1.547554e+12,16.0,0.0,16.0,5.0,0.0,0.0,0.0,855.0,inf,0.0,none,265900
3670,229145994,1735937.0,input,undefined,It doesn't match,1.547554e+12,17.0,1.0,16.0,0.0,0.0,0.0,0.0,5.0,5.000000,1.0,none,265900
